# Testing Uproot

uproot pack with xrootd read capability

In [2]:
import pyspark.sql

sqlContext = pyspark.sql.SparkSession.builder \
    .master('spark://nodo1.novalocal:7077') \
    .appName('Partition-test-uproot') \
    .config('spark.jars.packages', \
            'org.diana-hep:spark-root_2.11:0.1.16,org.diana-hep:histogrammar-sparksql_2.11:1.0.4') \
    .config('spark.driver.extraClassPath','/opt/hadoop/share/hadoop/common/lib/EOSfs.jar') \
    .config('spark.driver.extraClassPath','/home/shoh/hadoop-xrootd/hadoop-xrootd-1.0.4-jar-with-dependencies.jar') \
    .config('spark.executor.extraClassPath','/opt/hadoop/share/hadoop/common/lib/EOSfs.jar') \
    .config('spark.serializer','org.apache.spark.serializer.KryoSerializer') \
    .config('spark.sql.execution.arrow.enabled', "true") \
    .config('spark.eventLog.enabled','true') \
    .config('spark.eventLog.dir','/home/shoh/Evt_logger') \
    .config('spark.ui.enabled','true') \
    .config('spark.cores.max', '10') \
    .config('spark.executor.instances', '5') \
    .config('spark.executor.cores','2') \
    .config('spark.executor.memory','2g') \
    .getOrCreate()
    
print 'Spark version: ',sqlContext.version
print 'SparkSQL sesssion created'

Spark version:  2.4.3
SparkSQL sesssion created


## Uproot

In [ ]:
import uproot
import pandas as pd
#dsPath="root://eospublic.cern.ch//eos/opstest/cmspd-bigdata/DYJetsToLL_M-50_TuneCUETP8M1_13TeV-madgraphMLM-pythia8_ext1-v2.root" 
#dsPath="root://eosuser.cern.ch//eos/cms/store/group/phys_higgs/cmshww/amassiro/HWWNano/Summer16_102X_nAODv4_Full2016v5/MCl1loose2016v5/nanoLatino_ZZTo4L__part6.root"

#events = uproot.open(dsPath)["Events"]
files = {}
param = {}
files['VV'] = uproot.open('data/ntuple_4mu_VV.root')['HZZ4LeptonsAnalysisReduced']
files['bkg'] = uproot.open('data/ntuple_4mu_bkg.root')['HZZ4LeptonsAnalysisReduced']
param['VV'] = files['VV'].arrays()
param['bkg'] = files['bkg'].arrays()

df = {}
VARS = ['f_mass4l', 'f_massjj']
df['bkg'] = pd.DataFrame(param['bkg'],columns=VARS)
df['VV'] = pd.DataFrame(param['VV'],columns=VARS)

In [ ]:
sqlDF = {}

sqlDF['bkg'] = sqlContext.createDataFrame(df['bkg'])
sqlDF['VV'] = sqlContext.createDataFrame(df['VV'])

In [ ]:
sqlDF['bkg'].show()

### Using Uproot to partition


In [ ]:
sqlDF['bkg'].rdd.getNumPartitions()

In [ ]:
file.keys()

In [ ]:
dfnew.count()

## laurelin:0.3.0

In [ ]:
df1 = sqlContext.read.format('root') \
                .option("tree", "Events") \
                .load(dsPath)
df1.rdd.getNumPartitions()